In [1]:
from libs.alibaba.alibaba import Alibaba

from libs.json import JSON
from openpyxl import load_workbook

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq
from libs.CeleryTasks import tasks
import html
import time
import re
import os
import traceback
import types
import pendulum

In [2]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']

. storage markets.json
.\storage\markets.json was deserialized!


In [3]:
class Hot_Searched_Keywords:
    api = 'https://hz-mydata.alibaba.com/industry/keywords.htm'
    browser = None
    kws = set()
    
    def __init__(self, market, headless=True, browser=None):
        self.market = market
        self.lid = market['lid']
        self.lpwd = market['lpwd']
        self.config_dir = self.market['directory']+'_config'

        self.alibaba = None
        self.browser = browser
        self.headless = headless
        
    def login(self):
        self.alibaba = Alibaba(self.lid, self.lpwd, headless=self.headless, browser=self.browser)
        self.alibaba.login()
        self.browser = self.alibaba.browser
        
    def switch_to_keyword(self, keyword):
        input_keyword = self.browser.find_element_by_css_selector('#J-search-keywords')
        input_keyword.send_keys(Keys.CONTROL, 'a')
        input_keyword.send_keys(keyword)
        input_keyword.send_keys(Keys.RETURN)
        
    def crawl_current_page_keywords(self, keywords):
    #     WebDriverWait(self.browser, 15).until(
    #         EC.visibility_of_element_located((By.CSS_SELECTOR, "#J-tip-loading")))

        WebDriverWait(self.browser, 15).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, '#J-tip-loading')))
        time.sleep(0.5)


        table = self.browser.find_element_by_css_selector('#J-keywords-table')

        html = pq(table.get_attribute('innerHTML'))
        trs = html.find('tr.J-keyword-line')
    #         trs = self.browser.find_elements_by_css_selector('#J-keywords-content > tr')
        for idx, tr in enumerate(trs):
            tr = pq(tr)
            item = {}
            td = tr.find('td.column-keyword')
            key = td.text().strip()
            
            if key in self.kws:
                continue
            else:
                self.kws.add(key)
                
            kw = re.sub('eyelashes', 'eyelash', key)
            kw = re.sub('lashes', 'lash', kw)
            kw = re.sub('extensions', 'extension', kw)
            kw = re.sub('strips', 'strip', kw)
            kw = re.sub('boxes', 'box', kw)
            kw = re.sub('tweezers', 'tweezer', kw)
            kw = re.sub('samples', 'sample', kw)
            kw = re.sub('suppliers', 'supplier', kw)
            kw = re.sub('distributors', 'distributor', kw)
            kw = re.sub('brands', 'brand', kw)
            kw = re.sub('fans', 'fan', kw)
            kw = re.sub('clusters', 'cluster', kw)
            kw = re.sub('i beauty', 'i-beauty', kw)

            item['keyword'] = kw
            td = td.next()
            item['supplier_competition'] = int(td.text().strip())
            td = td.next()
            item['showroom_count'] = int(td.text().strip())
            td = td.next()
            item['search_frequency'] = int(td.text().strip())

            td = td.next()
            item['search_frequency_history'] = [ int(x) for x in td.text().split('\n')]

            if key not in keywords:
                keywords[key] = item
                
    def next_page(self):
        next_btn = self.browser.find_elements_by_css_selector('#J-pagination .ui-pagination-active + a')
        if next_btn:
            next_btn[0].click()
            return True
        else:
            return False

    def crawl_keywords(self, base_keywords):
        hotSearchedKeywords = {}
        self.kws.clear()
        for keyword in base_keywords:
            print(keyword, end=",")
            hskws.switch_to_keyword(keyword)
            while True:
                hskws.crawl_current_page_keywords(hotSearchedKeywords)
                if not hskws.next_page():
                    break
        
        kws = {}
        for key in hotSearchedKeywords:
            kw = hotSearchedKeywords[key]['keyword']
            if kw in kws:
                print(kws[kw])
                print(hotSearchedKeywords[key], key)
                kws[kw]['supplier_competition'] += hotSearchedKeywords[key]['supplier_competition']
                kws[kw]['showroom_count'] += hotSearchedKeywords[key]['showroom_count']
                kws[kw]['search_frequency'] += hotSearchedKeywords[key]['search_frequency']
                for index, value in enumerate(hotSearchedKeywords[key]['search_frequency_history']):
                    kws[kw]['search_frequency_history'][index] += hotSearchedKeywords[key]['search_frequency_history'][index]
                kws[kw]['keywords'].append(key)
                print(kws[kw])
                print('-------------------------------')
            else:
                kws[kw] = hotSearchedKeywords[key]
                kws[kw]['keywords'] = [key]
        
        old_hskw_list = hskws.deserialize()
        old_hskws = {}
        for kw in old_hskw_list:
            old_hskws[kw['keyword']] = kw
        
        new_kw_list = []
        kw_list = []
        for kw in kws:
            kw_list.append(kws[kw])
            #save new kws
            if kw not in old_hskws:
                new_kw_list.append(kws[kw])
        
        old_hskw_list = hskws.deserialize()
        for kw in old_hskw_list:
            # update with old results
            if kw['keyword'] not in kws:
                kw_list.append(kw)
        
        # sort and save the resulty
        sorted_kw_list = sorted(kw_list, key=lambda k: k['search_frequency'], reverse=True)
        self.serialize(sorted_kw_list)
        
        sorted_new_kw_list = sorted(new_kw_list, key=lambda k: k['search_frequency'], reverse=True)
        return sorted_new_kw_list
    
    def serialize(self, data):
        JSON.serialize(data, self.config_dir, '', 'hot_searched_keywords.json')
    
    def deserialize(self):
        return JSON.deserialize(self.config_dir, '', 'hot_searched_keywords.json')

In [135]:
hskws.crawl_current_page_keywords = types.MethodType(crawl_current_page_keywords, hskws)

In [4]:
hskws = Hot_Searched_Keywords(market, headless=False)

In [6]:
hskws.login()
hskws.browser.get(hskws.api)

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


In [ ]:
base_kws = ['eyelash', 'eyelashes', 'lash', 'lashes', 'volume tweezer', 'volume tweezers', 'pre fan', 'pre fans', 'premade fans']
base_kws.append('russian volume')
base_kws.append('russia volume')
base_kws.append('mega volume')
new_kw_list = hskws.crawl_keywords(base_kws)

eyelash,eyelashes,lash,

In [7]:
len(new_kw_list)

28

In [29]:
for kw in new_kw_list:
    print(kw['supplier_competition'], end='\t')
    print(kw['showroom_count'], end='\t')
    print(kw['search_frequency'], end='\t')
    print(kw['keyword'])
    

57	5	837	fake eyelash manufacturers
83	3	364	eyelash pads
84	8	325	colorful lash
83	2	317	eyelash extension trays
19	1	317	lash wands
13	1	307	lash extension trays
28	0	263	lash extention
8	0	261	false eyelash makeup
4	0	255	eyelash curler beauty tools
60	1	254	eyelash scissors
17	0	238	lash perming kit
0	0	231	eyelash extention supply
10	0	229	lash accessory
29	0	224	strip eye lash
8	0	220	i-beauty korea eyelash extension
2	0	213	individual eyelash extension tray
45	3	196	fan lash
236	16	192	colored eyelash extension
15	0	186	lash cleaning brushes
11	1	186	lash 5 pair
14	1	173	lift lash
32	1	172	eyelash extension gel pad
3	0	169	loose volume lash
2	0	164	extension eyelash tool
25	1	164	mink eyelash extentions
31	2	163	matt eyelash extension
5	0	163	primer for eyelash
5	3	163	lash lifting set
59	4	161	eyelash mascara brushes
238	10	161	natural falses eyelash
0	0	160	lash barbara
2	0	158	eyelash pack of 10
69	7	153	russian volume eyelash extension
4	0	153	own brand faux mink lash
0	0	15

0	0	55	eyelash decals
47	2	55	eyelash private label packaging
1	0	55	clear plastic eyelash case
0	0	55	eyelash organiser
28	1	55	magnetic box eyelash
1	0	55	eyelash extension glue 5 ml
8	1	55	mink eyelash cluster
0	0	55	candy eyelash
19	2	55	silicone lash pad
1	0	55	17mm lash
1	0	55	lash glue tray
0	0	55	display stand lash
0	0	55	lash necklace
12	0	55	lash fiber
0	0	55	lash volume lift
0	0	55	lash tray custom
1	0	55	minl lash
0	0	55	momo lash
2	0	55	mink lash with logo
0	0	55	mermaid lash extension
2	0	55	lash permanent
6	0	55	mink siberian lash
0	0	55	andrea lash
0	0	55	lash new
0	0	55	lamp for lash
3	0	55	remy lash wholesale
20	1	55	premade fan 3d 4d 5d 6d 8d russian volume lash
0	0	50	world beauty lash 3d mink eyelash
0	0	50	eyelash apparel
0	0	50	eyelash extension cleanser foam
1	0	50	eyelash tint kit
8	0	50	fiber eyelash mascara
1	0	50	eyelash ectension
2	0	50	acrylic eyelash tray
0	0	50	empty eyelash glue container
2	0	50	material eyelash
0	0	50	crown eyelash
16	0	50	empty eyelas

In [31]:
%store new_kw_list

Stored 'new_kw_list' (list)


In [32]:
%store -r new_kw_list

4732

In [38]:
%store hotSearchedKeywords

Stored 'hotSearchedKeywords' (dict)


In [1]:
%store -r hotSearchedKeywords

In [10]:
file = 'hot_searched_keywords.xlsx'
wb = load_workbook(file)
sheet = wb.worksheets[0]
sheet.cell(row=1,column=1).value = '买家竞争度'
sheet.cell(row=1,column=2).value = '橱窗数'
sheet.cell(row=1,column=3).value = '搜索热度'
sheet.cell(row=1,column=4).value = '关键字'

for idx, kw in enumerate(sorted_kw_list):
    row = idx+2
    sheet.cell(row=row,column=1).value = kw['supplier_competition']
    sheet.cell(row=row,column=2).value = kw['showroom_count']
    sheet.cell(row=row,column=3).value = kw['search_frequency']
    sheet.cell(row=row,column=4).value =  kw['keyword']
wb.save(file)

In [9]:
main_keywords_groups = {'general_keywords':[], 'mink_eyelash':[], 'silk_eyelash':[], 'magnetic_eyelash':[], 'eyelash_extension':[], 'markup_tools':[], 'packaging_box':[]}
main_keywords_groups['general_keywords'] = ['eyelash', 'lash']
main_keywords_groups['mink_eyelash'] = ['mink eyelash', 'mink lash', 'eyelash mink', 'lash mink']
main_keywords_groups['magnetic_eyelash'] = ['magnetic eyelash', 'magnetic lash']
main_keywords_groups['eyelash_extension'] = ['eyelash extension', 'lash extension']

one_words = ['faux mink', 'real mink', 'private label']

In [31]:
brands = ['koko']
for idx, kw in enumerate(sorted_kw_list):
    row = idx+2
    keyword = kw['keyword']
    if 'brands' in keyword:
        print(kw['supplier_competition'], '\t', kw['showroom_count'], '\t', kw['search_frequency'], '\t', keyword)

12 	 0 	 50 	 eyelash extension brands
